In [1]:
import os
import torch
import json
import numpy as np
from tqdm import tqdm

/home/zbyang/miniconda3/envs/cuda11.7/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
semSS_strings = np.load('/data/add_disk0/zbyang/datasets/coco_search18/semantic_seq_full/test.pkl', allow_pickle=True)

In [22]:
a = np.load('/data/add_disk0/zbyang/datasets/coco_search18/semantic_seq_full/test_TA_Sem.pkl', allow_pickle=True)
b = np.load('/data/add_disk0/zbyang/datasets/coco_search18/semantic_seq_full/test_TP_Sem.pkl', allow_pickle=True)

In [23]:
aa = {}
for k, v in a.items():
    x = []
    for y in v:
        x.append([yy[0] for yy in y])
    aa[k] = x
bb = {}
for k, v in b.items():
    x = []
    for y in v:
        x.append([yy[0] for yy in y])
    bb[k] = x

In [27]:
cc = aa.copy()
cc.update(bb)
len(aa),len(bb),len(cc)

(612, 612, 1224)

In [35]:
import pickle
with open('/data/add_disk0/zbyang/datasets/coco_search18/semantic_seq_full/test.pkl', 'wb') as f:
    pickle.dump(cc, f)

In [57]:
with open(os.path.join('/home/zbyang/coco_data_release/COCOFreeView-fixations.json')) as json_file:
    fv_sps = json.load(json_file)
len(fv_sps)

61839

In [58]:
img_names_train = np.unique([x['name'] for x in fv_sps if x['split'] == 'train'])
img_names_valid = np.unique([x['name'] for x in fv_sps if x['split'] == 'valid'])
img_names_test = np.unique([x['name'] for x in fv_sps if x['split'] == 'test'])
print(len(img_names_train), len(img_names_valid), len(img_names_test))
print(len(set(img_names_train) & set(img_names_valid)), len(set(img_names_train) & set(img_names_test)), len(set(img_names_valid) & set(img_names_test)))

3714 623 1127
20 32 97


In [59]:
# Remove the images that are in both train and valid/test
new_fv_sps = []
for sp in tqdm(fv_sps):
    sp.pop('task')
    if sp['split'] == 'valid' and sp['name'] in (set(img_names_train) & set(img_names_valid)):
        continue
    if sp['split'] == 'test' and sp['name'] in (set(img_names_train) & set(img_names_test)):
        continue
    if sp['split'] == 'test' and sp['name'] in (set(img_names_valid) & set(img_names_test)):
        continue
    new_fv_sps.append(sp)
len(new_fv_sps)

100%|██████████| 61839/61839 [00:23<00:00, 2619.58it/s]


60262

In [60]:
# deduplication
sps = {}
for sp in tqdm(new_fv_sps):
    idf = f'{sp["name"]}_{sp["subject"]}'
    sps[idf] = sp
len(new_fv_sps), len(sps)

100%|██████████| 60262/60262 [00:00<00:00, 1280677.88it/s]


(60262, 53015)

In [61]:
new_fv_sps = list(sps.values())
len(new_fv_sps)

53015

In [62]:
# Double check overlapping
img_names_train = np.unique([x['name'] for x in new_fv_sps if x['split'] == 'train'])
img_names_valid = np.unique([x['name'] for x in new_fv_sps if x['split'] == 'valid'])
img_names_test = np.unique([x['name'] for x in new_fv_sps if x['split'] == 'test'])
print(len(img_names_train), len(img_names_valid), len(img_names_test))
print(len(set(img_names_train) & set(img_names_valid)), len(set(img_names_train) & set(img_names_test)), len(set(img_names_valid) & set(img_names_test)))

3714 603 1000
0 0 0


In [63]:
trainval_split = list(filter(lambda x: x['split'] != 'test', new_fv_sps))
test_split = list(filter(lambda x: x['split'] == 'test', new_fv_sps))
print(len(trainval_split), len(test_split))

43048 9967


In [64]:
with open('/home/zbyang/coco_data_release/COCOFreeView_fixations_trainval_new.json', 'w') as json_file:
    json.dump(trainval_split, json_file, indent=4)
with open('/home/zbyang/coco_data_release/COCOFreeView_fixations_test_new.json', 'w') as json_file:
    json.dump(test_split, json_file, indent=4)

In [67]:
# Create a new split for COCO-Search18 according to the new train/valid/test split in COCOFreeView
with open(os.path.join('/home/zbyang/coco_data_release/coco_search18_fixations_TA_test.json')) as json_file:
    ta_test = json.load(json_file)
with open(os.path.join('/home/zbyang/coco_data_release/coco_search18_fixations_TA_trainval.json')) as json_file:
    ta_trainval = json.load(json_file)
    ta_train = list(filter(lambda x: x['split'] == 'train', ta_trainval))
    ta_valid = list(filter(lambda x: x['split'] == 'valid', ta_trainval))
print(len(ta_train), len(ta_valid), len(ta_test))

21627 3259 6120


In [68]:
new_ta_train, new_ta_valid, new_ta_test = [], [], []
for sp in ta_train + ta_valid + ta_test:
    if sp['name'] in img_names_train:
        new_ta_train.append(sp)
    elif sp['name'] in img_names_valid:
        new_ta_valid.append(sp)
    elif sp['name'] in img_names_test:
        new_ta_test.append(sp)
print(len(new_ta_train), len(new_ta_valid), len(new_ta_test))

21947 3789 5270


In [72]:
# Check for tasks
print(len(np.unique([x['task'] for x in new_ta_train])))
print(len(np.unique([x['task'] for x in new_ta_valid])))
print(len(np.unique([x['task'] for x in new_ta_test])))

18
18
18


In [19]:
a = torch.load('/data/add_disk0/zbyang/projects/scanpath_prediction_all/assets/pretrain_HAT/ckp_4999.pt')

In [22]:
a['model'].keys()


odict_keys(['encoder.backbone.stem.conv1.weight', 'encoder.backbone.stem.conv1.norm.weight', 'encoder.backbone.stem.conv1.norm.bias', 'encoder.backbone.stem.conv1.norm.running_mean', 'encoder.backbone.stem.conv1.norm.running_var', 'encoder.backbone.stages.res2.0.shortcut.weight', 'encoder.backbone.stages.res2.0.shortcut.norm.weight', 'encoder.backbone.stages.res2.0.shortcut.norm.bias', 'encoder.backbone.stages.res2.0.shortcut.norm.running_mean', 'encoder.backbone.stages.res2.0.shortcut.norm.running_var', 'encoder.backbone.stages.res2.0.conv1.weight', 'encoder.backbone.stages.res2.0.conv1.norm.weight', 'encoder.backbone.stages.res2.0.conv1.norm.bias', 'encoder.backbone.stages.res2.0.conv1.norm.running_mean', 'encoder.backbone.stages.res2.0.conv1.norm.running_var', 'encoder.backbone.stages.res2.0.conv2.weight', 'encoder.backbone.stages.res2.0.conv2.norm.weight', 'encoder.backbone.stages.res2.0.conv2.norm.bias', 'encoder.backbone.stages.res2.0.conv2.norm.running_mean', 'encoder.backbone.s

In [4]:
with open('/data/add_disk0/zbyang/datasets/refcoco/refcocogaze_train_correct_512X320.json') as f:
    data = json.load(f)

In [15]:
Xs.max(), Xs.min(), Ys.max(), Ys.min()

(tensor(507), tensor(7), tensor(320), tensor(0))

In [11]:
Xs, Ys = [], []
for d in data:
    Xs.append(torch.tensor(d['FIX_X']))
    Ys.append(torch.tensor(d['FIX_Y']))
Xs = torch.cat(Xs)
Ys = torch.cat(Ys)

In [16]:
torch.unique

<function torch.functional.boolean_dispatch.<locals>.fn(*args, **kwargs)>